In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tqdm
import seaborn as sns

import torch
import cv2
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
from pathlib import Path
from PIL import Image

In [ ]:
margin =20
image_size = 160

# Load face detector
mtcnn = MTCNN(keep_all=False, select_largest=False, post_process=False,
              device=device,margin=margin, image_size=image_size,thresholds=[0.7, 0.7, 0.7])

# Load facial recognition model
resnet = InceptionResnetV1(pretrained='casia-webface', device=device).eval()

In [ ]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # detect faces in the image
    face_array = mtcnn(pixels)
    if face_array==None:
        return None
    return face_array.permute(1,2,0).int().numpy()


In [ ]:
image = Image.open("/home/server/Shahid/Datasets/Bollywood_celeb_dataset_100_classes/bollywood_celeb_faces_0/Aamir_Khan/5.jpg")
image.size

In [ ]:
def load_face(dir):
    faces = list()
    # enumerate files
    for filename in os.listdir(dir):
        path = dir + '/'+ filename
        if (".jpg" not in filename) and (".png" not in filename):
            continue
        face = extract_face(path)
        if face is None:
            continue
        faces.append(face) 
    return faces

def load_dataset(dir):
    # list for faces and labels
    X, y = list(), list()
    for subdir in os.listdir(dir):
        path =dir +'/'+subdir
        faces = load_face(path)
        labels = [subdir for i in range(len(faces))]
        print("loaded %d sample for class: %s" % (len(faces),subdir) ) # print progress
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)


# load train dataset
trainX1, trainy1 = load_dataset('/home/server/Shahid/Datasets/Bollywood_celeb_dataset_100_classes/bollywood_celeb_faces_0')
print(trainX1.shape, trainy1.shape)

trainX2, trainy2 = load_dataset('/home/server/Shahid/Datasets/Bollywood_celeb_dataset_100_classes/bollywood_celeb_faces_1')
print(trainX2.shape, trainy2.shape)

trainX3, trainy3 = load_dataset('/home/server/Shahid/Datasets/Bollywood_celeb_dataset_100_classes/bollywood_celeb_faces2')
print(trainX3.shape, trainy3.shape)

# save and compress the dataset for further use
#np.savez_compressed('5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

In [ ]:
trainX=np.concatenate((trainX1,trainX2,trainX3), axis=0)
trainy=np.concatenate((trainy1,trainy2,trainy3), axis=0)

In [ ]:
trainy.shape

In [ ]:
np.savez_compressed('100_Bollywood_celebrity_faces_dataset.npz',trainX,trainy)

In [ ]:
# load the face dataset
data = np.load('100_Bollywood_celebrity_faces_dataset.npz')
trainX, trainy = data['arr_0'], data['arr_1']
print('Loaded: ', trainX.shape, trainy.shape)

In [ ]:
trainy.shape

In [ ]:
from sklearn.model_selection import train_test_split

trainX,testX,trainy,testy=train_test_split(trainX,trainy, test_size=0.33, random_state=42,stratify=trainy)

In [ ]:
from torchvision.transforms import ToTensor
tf_img = lambda i: ToTensor()(i).unsqueeze(0).float()
embeddings = lambda input: resnet(input)

In [ ]:
# convert each face in the train set into embedding
emdTrainX = list()
for face in trainX:
    #mean, std = face.mean(), face.std()
    #face = (face-mean)/std
    face=face/255
    t = tf_img(face).to(device)
    emd = embeddings(t).squeeze().cpu().tolist()
    emdTrainX.append(emd)
    #print(len(emd))
    
emdTrainX = np.asarray(emdTrainX)
print(emdTrainX.shape)

# convert each face in the test set into embedding
emdTestX = list()
for face in testX:
    mean, std = face.mean(), face.std()
    #face = (face-mean)/std
    face=face/255
    t = tf_img(face).to(device)
    emd = embeddings(t).squeeze().cpu().tolist()
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
print(emdTestX.shape)

# save arrays to one file in compressed format
np.savez_compressed('100-Bollywood_celebrity-faces-embeddings.npz', emdTrainX, trainy, emdTestX, testy)

In [ ]:
data=np.load("100-Bollywood_celebrity-faces-embeddings.npz")

In [ ]:
emdTrainX, trainy, emdTestX, testy=data["arr_0"],data["arr_1"],data["arr_2"],data["arr_3"]

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

print("Dataset: train=%d, test=%d" % (emdTrainX.shape[0], emdTestX.shape[0]))
# normalize input vectors
in_encoder = Normalizer()
emdTrainX_norm=emdTrainX#no effect of normalizer
emdTestX_norm=emdTestX
#emdTrainX_norm = in_encoder.transform(emdTrainX)
#emdTestX_norm = in_encoder.transform(emdTestX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy_enc = out_encoder.transform(trainy)
testy_enc = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(emdTrainX_norm, trainy_enc)
# predict
yhat_train = model.predict(emdTrainX_norm)
yhat_test = model.predict(emdTestX_norm)
# score
score_train = accuracy_score(trainy_enc, yhat_train)
score_test = accuracy_score(testy_enc, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

In [ ]:
from random import choice
# select a random face from test set
selection = choice([i for i in range(testX.shape[0])])
random_face = testX[selection]
random_face_emd = emdTestX_norm[selection]
random_face_class = testy_enc[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])

# prediction for the face
samples = np.expand_dims(random_face_emd, axis=0)
yhat_class = model.predict(samples)
yhat_prob = model.predict_proba(samples)
# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
all_names = out_encoder.inverse_transform(list(np.unique(trainy_enc)))
#print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
#print('Predicted: \n%s \n%s' % (all_names, yhat_prob[0]*100))
print('Expected: %s' % random_face_name[0])
print("predicted:",predict_names[0])
# plot face
plt.imshow(random_face)
title = '%s (%.3f)' % (predict_names[0], class_probability)
plt.title(title)
plt.show()